## 基于LangChain和Llama-3.1搭建RAG(检索增强生成)

### 环境准备

In [ ]:
# UBUNTU 22.04
# CUDA VERSION = 12.2
# NVDIA GPU > 24GB

### 环境配置

新建Conda环境：

conda create --name rag_langchain python=3.10

激活刚刚新建的Conda环境：

conda activate rag_langchain

接下来配置部署LLM的工具，这里我选择了MS-SWIFT[2]

首先下载源码（服务器网络环境不佳的可以先下载的自己的机器然后再上传到服务器并解压）：

git clone https://github.com/modelscope/swift.git

进入路径并安装：

cd ./your_path/swift

pip install -e .[llm]

拷贝LLM权重，国内推荐从魔塔社区modelscope去下载  

modelscope download --model=LLM-Research/Meta-Llama-3.1-8B-Instruct --local_dir Meta-Llama-3.1-8B-Instruct

向量数据库和langchain的安装

pip install langchain openai weaviate-client

至此，项目所需的环境已经配置完毕。

### 下载embedding模型

设置huggingface 国内环境变量

In [21]:
# !pip install -U huggingface_hub
# !export HF_ENDPOINT=https://hf-mirror.com

下载模型

In [22]:
# ! huggingface-cli download --local-dir-use-symlinks False --resume-download shibing624/text2vec-base-chinese --local-dir shibing624/text2vec-base-chinese

### 数据准备与存储向量数据库

这里我从网上摘录了一些关于RAG的知识保存在文件langchain_knowledge.txt中，内容如下：

RAG Architecture
典型的 RAG 应用程序有两个主要组件：

索引（Indexing）
用于从源获取数据并为其建立索引的管道。这通常发生在离线状态。

提取和生成（Retriever and generation）
实际的 RAG 链，它在运行时接受用户查询并从索引中检索相关数据，然后将其传递给模型。

索引（Indexing）

Load
首先需要加载数据，通过DocumentLoaders完成

Split
Text splitters将large Documents分成更小的chunks。这对于索引数据和将其传递到模型都很有用，因为大块更难搜索并且不适合模型的有限上下文窗口。

Store
存储和索引我们的分割，这通常是使用 VectorStore 和 Embeddings 模型来完成的。

#### 加载文档数据：

In [2]:
from langchain.document_loaders import TextLoader
txt_file_path = "/home/taoxu/test02/taoxu/101-Rag/data/langchain_knowledge.txt"
loader = TextLoader(txt_file_path)
document_raw = loader.load()
print(document_raw)

[Document(metadata={'source': '/home/taoxu/test02/taoxu/101-Rag/data/langchain_knowledge.txt'}, page_content='LangChain\nLangChain是一个软件开发框架，可以更轻松地使用大型语言模型（LLM）创建应用程序。它是一个具有 Python 和 JavaScript 代码库的开源工具。LangChain 允许开发人员将 GPT-4 等 LLM 与外部数据相结合，为聊天机器人、代码理解、摘要等各种应用程序开辟了可能性。\n\nLangChain模块\nLangChain将其功能分组到以下模块中：\n\n模型\n提示\n链\n代理\n记忆\n文档加载程序和索引\n提示\n        提示是指模型输入。在前面的部分中，您将提示硬编码为 LLM 和聊天模型。此技术不适用，因为在生产环境中不会收到硬编码的完整文本提示。相反，您将收到来自用户的简洁输入，您将希望将其转换为提示。\n\n模型\nLangChain支持三种类型的模型：\n\n大型语言模型\n聊天模型\n文本嵌入模型\n链\n链允许您同时运行多个LangChain模块。例如，使用链，您可以同时运行提示符和 LLM，从而避免了首先格式化 LLM 模型的提示，然后使用模型在单独的步骤中执行它。\n\nLangChain支持三种主要类型的链：\n\n简单的 LLM 链\n顺序链\n定制链\n代理\nLangChain代理涉及LLM来执行以下步骤：\n\n根据用户输入或其先前的输出确定要执行的操作。\n执行操作。\n观察输出。\n重复前三个步骤，直到它尽其所能完成用户输入中定义的任务。\nRAG Architecture\n典型的 RAG 应用程序有两个主要组件：\n\n索引（Indexing）\n用于从源获取数据并为其建立索引的管道。这通常发生在离线状态。\n\n提取和生成（Retriever and generation）\n实际的 RAG 链，它在运行时接受用户查询并从索引中检索相关数据，然后将其传递给模型。\n\n索引（Indexing）\n\nLoad\n首先需要加载数据，通过DocumentLoaders完成\n\nSplit\nText splitters将large Documents分成

#### 使用CharacterTextSplitter来分割文本：

对文档进行切片，避免文档太长超过大模型最大输入长度

In [3]:
# 使用CharacterTextSplitter来分割文本，设置chunk_size大约为500，chunk_overlap为50
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(document_raw)
print(documents)

[Document(metadata={'source': '/home/taoxu/test02/taoxu/101-Rag/data/langchain_knowledge.txt'}, page_content='LangChain\nLangChain是一个软件开发框架，可以更轻松地使用大型语言模型（LLM）创建应用程序。它是一个具有 Python 和 JavaScript 代码库的开源工具。LangChain 允许开发人员将 GPT-4 等 LLM 与外部数据相结合，为聊天机器人、代码理解、摘要等各种应用程序开辟了可能性。\n\nLangChain模块\nLangChain将其功能分组到以下模块中：\n\n模型\n提示\n链\n代理\n记忆\n文档加载程序和索引\n提示\n        提示是指模型输入。在前面的部分中，您将提示硬编码为 LLM 和聊天模型。此技术不适用，因为在生产环境中不会收到硬编码的完整文本提示。相反，您将收到来自用户的简洁输入，您将希望将其转换为提示。\n\n模型\nLangChain支持三种类型的模型：\n\n大型语言模型\n聊天模型\n文本嵌入模型\n链\n链允许您同时运行多个LangChain模块。例如，使用链，您可以同时运行提示符和 LLM，从而避免了首先格式化 LLM 模型的提示，然后使用模型在单独的步骤中执行它。\n\nLangChain支持三种主要类型的链：'), Document(metadata={'source': '/home/taoxu/test02/taoxu/101-Rag/data/langchain_knowledge.txt'}, page_content='LangChain支持三种主要类型的链：\n\n简单的 LLM 链\n顺序链\n定制链\n代理\nLangChain代理涉及LLM来执行以下步骤：\n\n根据用户输入或其先前的输出确定要执行的操作。\n执行操作。\n观察输出。\n重复前三个步骤，直到它尽其所能完成用户输入中定义的任务。\nRAG Architecture\n典型的 RAG 应用程序有两个主要组件：\n\n索引（Indexing）\n用于从源获取数据并为其建立索引的管道。这通常发生在离线状态。\n\n提取和生成（Retriever and generation）\n实际的 RAG

#### 自己写一个文档

In [32]:
from langchain_core.documents import Document
documents = [
    Document(
        page_content="狗是很好的伴侣，以忠诚和友好著称。",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="猫是独立的宠物，猫常常喜欢自己的空间。",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="金鱼是初学者的热门宠物，护理相对简单。",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="鹦鹉是聪明的鸟类，能够模仿人类语言。",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="兔子是社会性动物，需要足够的空间跳跃。",
        metadata={"source": "mammal-pets-doc"},
    ),
]

#### 切片后的文本进行向量化，保存到向量数据库

In [33]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
BGE_MODEL_PATH = "/home/taoxu/test02/taoxu/cache/shibing624/text2vec-base-chinese"
huggingface_bge_embedding = HuggingFaceBgeEmbeddings(model_name=BGE_MODEL_PATH)
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents,
    embedding=huggingface_bge_embedding,
)

In [3]:
vectorstore.similarity_search("狗")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='狗是很好的伴侣，以忠诚和友好著称。'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='金鱼是初学者的热门宠物，护理相对简单。'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='兔子是社会性动物，需要足够的空间跳跃。'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='猫是独立的宠物，猫常常喜欢自己的空间。')]

In [4]:
await vectorstore.asimilarity_search("狗")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='狗是很好的伴侣，以忠诚和友好著称。'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='金鱼是初学者的热门宠物，护理相对简单。'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='兔子是社会性动物，需要足够的空间跳跃。'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='猫是独立的宠物，猫常常喜欢自己的空间。')]

In [5]:
vectorstore.similarity_search_with_score("狗")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='狗是很好的伴侣，以忠诚和友好著称。'),
  350.973388671875),
 (Document(metadata={'source': 'fish-pets-doc'}, page_content='金鱼是初学者的热门宠物，护理相对简单。'),
  400.5721130371094),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='兔子是社会性动物，需要足够的空间跳跃。'),
  405.5841064453125),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='猫是独立的宠物，猫常常喜欢自己的空间。'),
  424.0150146484375)]

In [7]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["狗", "金鱼"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='狗是很好的伴侣，以忠诚和友好著称。')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='金鱼是初学者的热门宠物，护理相对简单。')]]

### 大模型部署

在终端运行swift部署模型的命令：

CUDA_VISIBLE_DEVICES=0 swift deploy --model_id_or_path /home/taoxu/test02/taoxu/cache/Meta-Llama-3.1-8B-Instruct --model_type llama3_1-8b-instruct

其中CUDA_VISIBLE_DEVICES是用来指定模型加载到哪张卡上，单卡可以忽略不写

部署之后，可以用下面的代码来测试是否部署成功

In [29]:
from openai import OpenAI
client = OpenAI(
    api_key='EMPTY', #随便填
    base_url='http://localhost:8000/v1', #填部署成功后的地址+端口+v1
)
model_type = client.models.list().data[0].id
print(f'model_type: {model_type}')


model_type: llama3_1-8b-instruct


若模型正确部署，则会输出之前在终端指定的model_type值，例如这里是：model_type: llama3_1-8b-instruct

### 用openai的格式访问之前部署的大模型

In [30]:
import os
API_SECRET_KEY = "EMPTY"
BASE_URL = "http://localhost:8000/v1"

# 设置环境变量
os.environ["OPENAI_API_KEY"] = API_SECRET_KEY
os.environ["OPENAI_API_BASE"] = BASE_URL

from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="llama3_1-8b-instruct", temperature=0)


In [31]:
llm.invoke("请聊聊金鱼")

AIMessage(content='金鱼是一种常见的宠物鱼，人们喜欢养金鱼的原因有很多。首先，金鱼的颜色多样，种类繁多，人们可以根据自己的喜好选择喜欢的颜色和形状的金鱼。其次，金鱼的生长速度快，养起来比较容易，不需要太多的空间和照顾。再次，金鱼的价格便宜，人们可以在不花太多钱的情况下拥有自己的金鱼。最后，金鱼的观赏性强，人们可以在观赏金鱼的过程中放松身心，减压。', response_metadata={'token_usage': {'completion_tokens': 135, 'prompt_tokens': 15, 'total_tokens': 150}, 'model_name': 'llama3_1-8b-instruct', 'system_fingerprint': None, 'finish_reason': None, 'logprobs': None}, id='run-3edc1ed3-89fa-42f5-855b-38a40e3e0a90-0')

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
请根据我提供的内容回答问题.

{question}

提供的内容:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [35]:
response = rag_chain.invoke("请聊聊金鱼")

print(response.content)

金鱼是初学者的热门宠物，护理相对简单。
